# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [1]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

Populating the interactive namespace from numpy and matplotlib


In [2]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print(classes)

['Back', 'Belly', 'Crouch', 'Frog', 'HeadBack', 'Knee', 'Left', 'Right', 'Sit', 'Stand', 'StandInit']


In [3]:
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    data = pickle.load(open(filename, 'rb'))
    target = [i] * len(data)
    return data, target

In [23]:
# load all the data
all_data = []
all_target = []

for a_class in range(len(classes)):
    data_target = load_pose_data(a_class)
    all_data += data_target[0]
    all_target += data_target[1]

print('total number of data', len(all_data))

[-0.05058002471923828, -0.2269899845123291, -0.8298521041870117, 1.7180380821228027, -0.05058002471923828, -0.1487560272216797, -0.9311800003051758, 1.8945322036743164, -0.008992496877908707, -1.516520619392395]
[0.06600403785705566, 0.0061779022216796875, 0.07980990409851074, 2.0079641342163086, 0.06600403785705566, 0.0844118595123291, 0.09506607055664062, 1.8976001739501953, -0.018579784780740738, 1.4904650449752808]
[0.0061779022216796875, -0.03217196464538574, -0.9970581531524658, 2.1414220333099365, 0.0061779022216796875, 0.029187917709350586, -0.995607852935791, 2.1415061950683594, -0.012217044830322266, 0.07155847549438477]
[-0.40953612327575684, -0.19477605819702148, -1.579978108406067, 2.136820077896118, -0.40953612327575684, 0.2853660583496094, -1.6613640785217285, 2.13997220993042, -0.02792501449584961, 0.9896016120910645]
[0.2055978775024414, -0.23926210403442383, -0.2438640594482422, -0.03072190284729004, 0.2055978775024414, -0.13955211639404297, -4.1961669921875e-05, -0.1

In [5]:
# shuffule data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.7)
training_data = permutation[:n_training_data]

## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [6]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [7]:
train_x = []
train_y = []

for n in training_data:
    train_x.append(all_data[n])
    train_y.append(all_target[n])
    
clf.fit(train_x, train_y)

SVC(C=100.0, gamma=0.001)

### predicting

In [8]:
predicted, expected = clf.predict(all_data), all_target

In [9]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

## 4. Evaluate on the test data

In [10]:
evaluate(expected, predicted)

Classification report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        23
           1       1.00      1.00      1.00        19
           2       0.97      1.00      0.98        30
           3       1.00      1.00      1.00        10
           4       1.00      1.00      1.00        10
           5       1.00      1.00      1.00        10
           6       1.00      1.00      1.00        20
           7       1.00      0.91      0.95        11
           8       1.00      1.00      1.00        26
           9       1.00      1.00      1.00        11
          10       1.00      1.00      1.00        52

    accuracy                           1.00       222
   macro avg       1.00      0.99      0.99       222
weighted avg       1.00      1.00      1.00       222


Confusion matrix:
[[23  0  0  0  0  0  0  0  0  0  0]
 [ 0 19  0  0  0  0  0  0  0  0  0]
 [ 0  0 30  0  0  0  0  0  0  0  0]
 [ 0  0  0 10  0  0  0  0  0  0  0]
 

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [11]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'wb'))

Then, in the application we can load the trained classifier again.

In [12]:
clf2 = pickle.load(open(ROBOT_POSE_CLF, 'rb'))
clf2.predict(all_data), all_target

(array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,
         2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
         2,  2,  2,  2,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,  4,
         4,  4,  4,  4,  4,  4,  4,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
         6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,
         6,  6,  6,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  2,  8,  8,  8,
         8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
         8,  8,  8,  8,  8,  8,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10,
        10])